In [1]:
import pandas as pd
import requests
import os
import csv
import concurrent.futures

file_path = "list_gwas_summary_statistics.tsv"
data = pd.read_csv(file_path, sep='\t')

# Now, the 'data' variable contains the contents of the TSV file as a DataFrame

data["Data Access"][0]

'http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST003001-GCST004000/GCST003738'

In [2]:
def download_file(url):
    # Extract the GCST number from the URL and construct the filename
    gcst_number = url.split("/")[-1]

    # Construct the filename
    filename = f"{gcst_number}_buildGRCh37.tsv.gz"

    # Append the filename to the URL
    file_url = url + "/" + filename

    # Send a GET request to the URL
    response = requests.get(file_url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the file name from the URL
        file_name = os.path.join(save_dir, filename)

        # Open the file and write the content of the request to it
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
                
        print(f"Downloaded file: {file_name}")

# Directory to save files. In this case, it's the current directory.
save_dir = 'C:\\Users\\falty\\Desktop\\geometric-omics\\GWAS-Catalog\\data'

# Maximum number of files to download
max_files = 2

# Get the first max_files URLs
urls = data["Data Access"][:max_files]

# Create a ThreadPoolExecutor with a maximum of 12 threads
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
    # Start downloading files
    executor.map(download_file, urls)

Downloaded file: C:\Users\falty\Desktop\geometric-omics\GWAS-Catalog\data\GCST003738_buildGRCh37.tsv.gz
Downloaded file: C:\Users\falty\Desktop\geometric-omics\GWAS-Catalog\data\GCST003739_buildGRCh37.tsv.gz
